In [1]:
# Imports 

from helper import ClassConstructor
from db.players_controller import PlayersController
from db.clubs_controller import ClubsController
from db.stadiums_controller import StadiumsController
from db.games_controller import GamesController

from logs_helper import LogsHandler

# Just int case
from pprint import pprint

In [2]:
# Class declare (statics)

class_const = ClassConstructor()
players_controller = PlayersController()
clubs_controller = ClubsController()
stadiums_controller = StadiumsController()
games_controller = GamesController()

logs_handler = LogsHandler()

season = '2022'

In [3]:
# Select and transform stadium data into objects

stadiums_data = stadiums_controller.select_all_stadiums()
stadiums = class_const.stadiums(stadiums_data)

In [4]:
# Select players with contract

# get from db
players_data = players_controller.select_players_with_contract(season) 

In [5]:
# Serie A clubs selection & cast

serie_a = clubs_controller.select_serie_a_clubs(season)
serie_a_clubs = class_const.prepare_clubs(serie_a, players_data)

In [6]:
# Confirm that clubs are created

serie_a_clubs

[Club(América Mineiro),
 Club(Atlético Mineiro),
 Club(Athletico Paranaense),
 Club(Ceará),
 Club(Fortaleza),
 Club(Flamengo),
 Club(Fluminense),
 Club(Cuiabá),
 Club(Grêmio),
 Club(International),
 Club(Chapecoense),
 Club(Juventude),
 Club(São Paulo),
 Club(Corinthians),
 Club(Palmeiras),
 Club(Santos),
 Club(Red Bull Bragantino),
 Club(Sport Recife),
 Club(Bahia),
 Club(Atlético Goianiense)]

In [7]:
# Check for start eleven 

serie_a_clubs[0].start_eleven

[Player(Jaques Mica),
 Player(Enzo Volquez),
 Player(Caubi Brito),
 Player(Evaldo Nazaré),
 Player(Pericles Hoffman),
 Player(Grajaú Cazares),
 Player(Madge Neto),
 Player(David Gonçalves),
 Player(Antíope Falcao),
 Player(Natanael Silva),
 Player(Enrique Falquez)]

In [8]:
# Define the clubs matches, this will return a list of lists with two differents teams

serie_a_schedule = class_const.define_schedule(serie_a_clubs)

In [9]:
# Return a list of games object with all the confrontations through the season

serie_a_games = class_const.prepare_games(serie_a_schedule, stadiums, 'Campeonato Brasileiro Serie A', 1, int(season))

In [29]:
serie_a_games[5].start()

{'game_stats': {'América Mineiro': {'home': True,
   'goals': 0,
   'shots': 16,
   'shots on target': 12,
   'fouls': 25,
   'passes': 139,
   'wrong passes': 32,
   'interceptions': 37,
   'tackles': 16,
   'stolen_balls': 14,
   'saves': 10,
   'ball possession': 0,
   'offsides': 0,
   'free kicks': 0,
   'penalties': 0},
  'Fluminense': {'home': True,
   'goals': 1,
   'shots': 0,
   'shots on target': 11,
   'fouls': 45,
   'passes': 139,
   'wrong passes': 30,
   'interceptions': 20,
   'tackles': 15,
   'stolen_balls': 23,
   'saves': 12,
   'ball possession': 0,
   'offsides': 0,
   'free kicks': 0,
   'penalties': 0}},
 'scoreboard': '0 x 1',
 'player_stats': {'assists': defaultdict(int, {}),
  'goals': defaultdict(int, {Player(Ramão Mico): 1}),
  'tackles': defaultdict(int,
              {Player(Lucca Santos): 4,
               Player(Jaques Mica): 1,
               Player(Venâncio Silva): 2,
               Player(Herman Farid): 1,
               Player(Ramão Mico): 2,
     

In [11]:
# check for home team's overall 

serie_a_games[0].home.overall

75

In [12]:
# check for away team's overall 

serie_a_games[0].away.overall

85

In [13]:
print("CLUB\t\t\tOVERALL")
for club in serie_a_clubs:
    if len(club.name) <= 6:
        print(f"{club.name}\t\t\t{club.overall}")
    elif len(club.name) > 5 and len(club.name) <= 15:
        print(f"{club.name}\t\t{club.overall}")
    else:
        print(f"{club.name}\t{club.overall}")

CLUB			OVERALL
América Mineiro		75
Atlético Mineiro	85
Athletico Paranaense	75
Ceará			74
Fortaleza		75
Flamengo		84
Fluminense		84
Cuiabá			75
Grêmio			85
International		85
Chapecoense		65
Juventude		65
São Paulo		85
Corinthians		85
Palmeiras		84
Santos			75
Red Bull Bragantino	74
Sport Recife		65
Bahia			74
Atlético Goianiense	75


In [14]:
"""
TODO

adicionar mais uma partida a todos os jogadores que entraram na partida

adicionar clean_sheets aos defensores caso nao tenha rolado gols
adicionar goals_conceded aos defensores caso tenham sofrido gols


adicionar clearances ao algoritimo

melhorar o algoritimo para que tenha mais passes, desarmes e tackles durante as partidas
"""

'\nTODO\n\nadicionar mais uma partida a todos os jogadores que entraram na partida\n\nadicionar clean_sheets aos defensores caso nao tenha rolado gols\nadicionar goals_conceded aos defensores caso tenham sofrido gols\n\n\nadicionar clearances ao algoritimo\n\nmelhorar o algoritimo para que tenha mais passes, desarmes e tackles durante as partidas\n'

In [15]:
logs = serie_a_games[0].logs
game = serie_a_games[0]

In [16]:
stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(logs, game.home, game.away))

In [17]:
# Check for the stats data

stats_data

[[1, 11, 12, 29, 137, 25, 28, 15, 20, 13, 0, 0, 0, 0, 1],
 [0, 5, 13, 41, 168, 39, 31, 22, 17, 11, 0, 0, 0, 0, 2]]

In [18]:
# Insert game stats into database

s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])
s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])

In [19]:
# Get game stats ids

game_stats_ids = s1[0][0], s2[0][0]

In [20]:
# Prepare game data

prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_stats_ids)

In [21]:
# Get game id

game_id = games_controller.insert_game(prepare_game)

In [22]:
# After this is the preparation to insert player stats per game on databse

game_id = game_id[0][0]

In [23]:
player_stats = game.logs['stats']

In [24]:
player_stats['Hernán Garmendez']

KeyError: 'Hernán Garmendez'

In [ ]:
game.away.start_eleven

In [ ]:
from data_helper import prepare_player_stats_to_db

In [ ]:
player_stats_data = prepare_player_stats_to_db(player_stats, season, game_id)

In [ ]:
player_stats_data

In [ ]:
players_controller.insert_players_stats(player_stats_data)

In [ ]:
# Prepare a stats data for game_stats 
"""
for game in serie_a_games:
    stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(game.logs, game.home, game.away))

    # Insert and get the id from the game stats
    s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])
    s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])

    # get the last games_stats id inserted
    game_ids = s1[0][0], s2[0][0]

    # prepare data to insert game
    prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_ids)

    # insert games into the database
    games_controller.insert_games_list([prepare_game])
"""

In [ ]:
# Ajeitar o algoritimo pq goleiro ta chutando no alvo e no gol 
# 
#